In [1]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.naive_bayes
import sklearn.ensemble as ske
import sklearn.neighbors
import sklearn.pipeline
import sklearn.linear_model
import joblib
from sklearn import preprocessing
from sklearn import tree, linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel
import sys
import sqlite3
import importlib

In [2]:
df_wild = pd.read_pickle('/kaggle/input/wild-pickle/wild.pickle')

In [3]:
df = df_wild.sample(n=10000, random_state=102)

In [4]:
df = df.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
df = df.drop(['benign_vt','malicious_vt'], axis=1)

In [5]:
string_columns = df.select_dtypes(include=['object']).columns
df = df.drop(string_columns, axis=1)

In [6]:
df.head()

,malicious,packed,imp_setupremovefilelogentrya,imp_vardeccmp,imp_setdlgitemint,imp_?append@?$basic_string@du?$char_traits@d@std@@v?$allocator@d@2@@std@@qaeaav12@abv12@ii@z,imp_zwwritefile,imp_glstencilop,imp_comparelinkinforeferents,imp__setmbcp,...,opcode_xor sub mov,opcode_xor sub mov add,opcode_xor test,opcode_xor test je,opcode_xor xor,opcode_xor xor mov,opcode_xor xor pop,opcode_xor xor pop mov,opcode_xor xor xor,opcode_xor xor xor pop
167256,True,True,False,False,False,False,False,False,False,False,...,0.000087,0.0,0.000329,0.000267,0.000086,0.00000,0.0,0.0,0.000000,0.0
14422,True,True,False,False,False,False,False,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0
185351,True,True,False,False,False,False,False,False,False,False,...,0.000029,0.0,0.000105,0.000099,0.000085,0.00011,0.0,0.0,0.000043,0.0
53241,False,True,False,False,False,False,False,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0
38618,False,True,False,False,False,False,False,False,False,False,...,0.000000,0.0,0.000199,0.000124,0.000000,0.00000,0.0,0.0,0.000000,0.0


In [7]:
df_7000 = df.sample(n=7000, random_state=102)
df_3000 = df.drop(df_7000.index)

In [8]:
data = df_7000.sample(n=3500, random_state=102)

In [9]:
#Kiểm tra số lượng mẫu
UB = data[(data['malicious'] == False) & (data['packed'] == False)].shape[0]
PM = data[(data['malicious'] == True) & (data['packed'] == True)].shape[0]
PB = data[(data['malicious'] == False) & (data['packed'] == True)].shape[0]
print('Unpacked Benign: %i samples' % UB)
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (UB+PM+PB))

Unpacked Benign: 293 samples
Packed Malware: 2341 samples
Packed Benign: 866 samples
Total TRAIN set: 3500 samples


In [10]:
#Chia tỉ lệ 50 - 50
data_PM = data[(data['malicious'] == True) & (data['packed'] == True)].sample(n=(PM-(UB+PB)), random_state=102)
data = data.drop(data_PM.index)

In [11]:
#Số lượng tập train
UB = data[(data['malicious'] == False) & (data['packed'] == False)].shape[0]
PM = data[(data['malicious'] == True) & (data['packed'] == True)].shape[0]
PB = data[(data['malicious'] == False) & (data['packed'] == True)].shape[0]
print('Unpacked Benign: %i samples' % UB)
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (UB+PM+PB))

Unpacked Benign: 293 samples
Packed Malware: 1159 samples
Packed Benign: 866 samples
Total TRAIN set: 2318 samples


In [12]:
#Xử lý tập train
y = data['malicious'].values
data = data.drop(['malicious','packed'], axis=1)

In [13]:
UB = df_3000[(df_3000['malicious'] == False) & (df_3000['packed'] == False)].shape[0]
PM = df_3000[(df_3000['malicious'] == True) & (df_3000['packed'] == True)].shape[0]
PB = df_3000[(df_3000['malicious'] == False) & (df_3000['packed'] == True)].shape[0]

df_3000_PM = df_3000[(df_3000['malicious'] == True) & (df_3000['packed'] == True)].sample(n=(PM-(UB+PB)), random_state=102)
df_3000 = df_3000.drop(df_3000_PM.index)

UB = df_3000[(df_3000['malicious'] == False) & (df_3000['packed'] == False)].shape[0]
PM = df_3000[(df_3000['malicious'] == True) & (df_3000['packed'] == True)].shape[0]
PB = df_3000[(df_3000['malicious'] == False) & (df_3000['packed'] == True)].shape[0]

print('Unpacked Benign: %i samples' % UB)
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (UB+PM+PB))

Unpacked Benign: 254 samples
Packed Malware: 1022 samples
Packed Benign: 768 samples
Total TEST set: 2044 samples


In [14]:
# model = SelectFromModel(fsel, prefit=True)

X_train = data
X_test = df_3000.drop(['malicious','packed'], axis=1)

data = data.loc[:, (data != 0).any(axis=0)]
x_train = data
X_test = df_3000.drop(['malicious','packed'], axis=1)
y_train = y
y_test = df_3000['malicious'].values

In [15]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train, y_train)
random_forest_preds = random_forest.predict(X_test)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test))

The accuracy of the Random Forests model is :	 0.9256360078277887


In [16]:
mt = confusion_matrix(y_test, random_forest_preds)
report = classification_report(y_test, random_forest_preds)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 944 
True negative : 948 
False positive : 74 
False negative : 78 
Classification Report:
               precision    recall  f1-score   support

       False       0.92      0.93      0.93      1022
        True       0.93      0.92      0.93      1022

    accuracy                           0.93      2044
   macro avg       0.93      0.93      0.93      2044
weighted avg       0.93      0.93      0.93      2044



# **dolphin**

In [17]:
df_dolphin = pd.read_pickle('/kaggle/input/packer-balance-pickle/dolphin_balance.pickle')

In [18]:
df_dolphin = df_dolphin.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
df_dolphin = df_dolphin.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = df_dolphin.select_dtypes(include=['object']).columns
df_dolphin = df_dolphin.drop(string_columns, axis=1)

In [19]:
df_7000_dolphin = df_dolphin.sample(n=7000, random_state=102)
df_3000_dolphin = df_dolphin.drop(df_7000_dolphin.index)
data_dolphin = df_7000_dolphin.sample(n=3500, random_state=102)

In [20]:
#Kiểm tra số lượng mẫu
PM = data_dolphin[(data_dolphin['malicious'] == True) & (data_dolphin['packed'] == True)].shape[0]
PB = data_dolphin[(data_dolphin['malicious'] == False) & (data_dolphin['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_dolphin = data_dolphin[(data_dolphin['malicious'] == True) & (data_dolphin['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_dolphin = data_dolphin.drop(data_PM_dolphin.index)

#Số lượng tập train
PM = data_dolphin[(data_dolphin['malicious'] == True) & (data_dolphin['packed'] == True)].shape[0]
PB = data_dolphin[(data_dolphin['malicious'] == False) & (data_dolphin['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y_dolphin = data_dolphin['malicious'].values
data_dolphin = data_dolphin.drop(['malicious','packed'], axis=1)

PM = df_3000_dolphin[(df_3000_dolphin['malicious'] == True) & (df_3000_dolphin['packed'] == True)].shape[0]
PB = df_3000_dolphin[(df_3000_dolphin['malicious'] == False) & (df_3000_dolphin['packed'] == True)].shape[0]

df_3000_PM_dolphin = df_3000_dolphin[(df_3000_dolphin['malicious'] == True) & (df_3000_dolphin['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_dolphin = df_3000_dolphin.drop(df_3000_PM_dolphin.index)

PM = df_3000_dolphin[(df_3000_dolphin['malicious'] == True) & (df_3000_dolphin['packed'] == True)].shape[0]
PB = df_3000_dolphin[(df_3000_dolphin['malicious'] == False) & (df_3000_dolphin['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2322 samples
Packed Benign: 1178 samples
Total TRAIN set: 3500 samples
Packed Malware: 1178 samples
Packed Benign: 1178 samples
Total TRAIN set: 2356 samples
Packed Malware: 1022 samples
Packed Benign: 1022 samples
Total TEST set: 2044 samples


In [21]:
# model = SelectFromModel(fsel, prefit=True)

X_train_dolphin = data_dolphin
X_test_dolphin = df_3000_dolphin.drop(['malicious','packed'], axis=1)

# data_dolphin = data_dolphin.loc[:, (data_dolphin != 0).any(axis=0)]
x_train_dolphin = data_dolphin
X_test_dolphin = df_3000_dolphin.drop(['malicious','packed'], axis=1)
y_train_dolphin = y_dolphin
y_test_dolphin = df_3000_dolphin['malicious'].values

In [22]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train, y_train)
random_forest_preds = random_forest.predict(X_test_dolphin)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_dolphin))

The accuracy of the Random Forests model is :	 0.5


In [23]:
mt = confusion_matrix(y_test_dolphin, random_forest_preds)
report = classification_report(y_test_dolphin, random_forest_preds)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 1022 
True negative : 0 
False positive : 1022 
False negative : 0 
Classification Report:
               precision    recall  f1-score   support

       False       0.00      0.00      0.00      1022
        True       0.50      1.00      0.67      1022

    accuracy                           0.50      2044
   macro avg       0.25      0.50      0.33      2044
weighted avg       0.25      0.50      0.33      2044



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest_new = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest_new.fit(x_train_dolphin, y_train_dolphin)
random_forest_preds_new = random_forest_new.predict(X_test)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds_new,y_test))

The accuracy of the Random Forests model is :	 0.62573385518591


In [25]:
mt = confusion_matrix(y_test, random_forest_preds_new)
report = classification_report(y_test, random_forest_preds_new)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 382 
True negative : 897 
False positive : 125 
False negative : 640 
Classification Report:
               precision    recall  f1-score   support

       False       0.58      0.88      0.70      1022
        True       0.75      0.37      0.50      1022

    accuracy                           0.63      2044
   macro avg       0.67      0.63      0.60      2044
weighted avg       0.67      0.63      0.60      2044



# **kkrunchy**

In [26]:
kkrunchy = pd.read_pickle('/kaggle/input/packer-balance-pickle/kkrunchy_balance.pickle')

In [27]:
kkrunchy = kkrunchy.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
kkrunchy = kkrunchy.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = kkrunchy.select_dtypes(include=['object']).columns
kkrunchy = kkrunchy.drop(string_columns, axis=1)

In [28]:
df_7000_kkrunchy = kkrunchy.sample(n=7000, random_state=102)
df_3000_kkrunchy = kkrunchy.drop(df_7000_kkrunchy.index)
data_kkrunchy = df_7000_kkrunchy.sample(n=3500, random_state=102)

In [29]:
#Kiểm tra số lượng mẫu
PM = data_kkrunchy[(data_kkrunchy['malicious'] == True) & (data_kkrunchy['packed'] == True)].shape[0]
PB = data_kkrunchy[(data_kkrunchy['malicious'] == False) & (data_kkrunchy['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_kkrunchy = data_kkrunchy[(data_kkrunchy['malicious'] == True) & (data_kkrunchy['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_kkrunchy = data_kkrunchy.drop(data_PM_kkrunchy.index)

#Số lượng tập train
PM = data_kkrunchy[(data_kkrunchy['malicious'] == True) & (data_kkrunchy['packed'] == True)].shape[0]
PB = data_kkrunchy[(data_kkrunchy['malicious'] == False) & (data_kkrunchy['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y_kkrunchy = data_kkrunchy['malicious'].values
data_kkrunchy = data_kkrunchy.drop(['malicious','packed'], axis=1)

PM = df_3000_kkrunchy[(df_3000_kkrunchy['malicious'] == True) & (df_3000_kkrunchy['packed'] == True)].shape[0]
PB = df_3000_kkrunchy[(df_3000_kkrunchy['malicious'] == False) & (df_3000_kkrunchy['packed'] == True)].shape[0]

df_3000_PM_kkrunchy = df_3000_kkrunchy[(df_3000_kkrunchy['malicious'] == True) & (df_3000_kkrunchy['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_kkrunchy = df_3000_kkrunchy.drop(df_3000_PM_kkrunchy.index)

PM = df_3000_kkrunchy[(df_3000_kkrunchy['malicious'] == True) & (df_3000_kkrunchy['packed'] == True)].shape[0]
PB = df_3000_kkrunchy[(df_3000_kkrunchy['malicious'] == False) & (df_3000_kkrunchy['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2411 samples
Packed Benign: 1089 samples
Total TRAIN set: 3500 samples
Packed Malware: 1089 samples
Packed Benign: 1089 samples
Total TRAIN set: 2178 samples
Packed Malware: 913 samples
Packed Benign: 913 samples
Total TEST set: 1826 samples


In [30]:
# model = SelectFromModel(fsel, prefit=True)

X_train_kkrunchy = data_kkrunchy
X_test_kkrunchy = df_3000_kkrunchy.drop(['malicious','packed'], axis=1)

# data_kkrunchy = data_kkrunchy.loc[:, (data_kkrunchy != 0).any(axis=0)]
x_train_kkrunchy = data_kkrunchy
X_test_kkrunchy = df_3000_kkrunchy.drop(['malicious','packed'], axis=1)
y_train_kkrunchy = y_kkrunchy
y_test_kkrunchy = df_3000_kkrunchy['malicious'].values

In [31]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train, y_train)
random_forest_preds = random_forest.predict(X_test_kkrunchy)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_kkrunchy))

The accuracy of the Random Forests model is :	 0.5


In [32]:
mt = confusion_matrix(y_test_kkrunchy, random_forest_preds)
report = classification_report(y_test_kkrunchy, random_forest_preds)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 913 
True negative : 0 
False positive : 913 
False negative : 0 
Classification Report:
               precision    recall  f1-score   support

       False       0.00      0.00      0.00       913
        True       0.50      1.00      0.67       913

    accuracy                           0.50      1826
   macro avg       0.25      0.50      0.33      1826
weighted avg       0.25      0.50      0.33      1826



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest_new = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest_new.fit(x_train_kkrunchy, y_train_kkrunchy)
random_forest_preds_new = random_forest_new.predict(X_test)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds_new,y_test))

The accuracy of the Random Forests model is :	 0.75


In [34]:
mt = confusion_matrix(y_test, random_forest_preds_new)
report = classification_report(y_test, random_forest_preds_new)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 605 
True negative : 928 
False positive : 94 
False negative : 417 
Classification Report:
               precision    recall  f1-score   support

       False       0.69      0.91      0.78      1022
        True       0.87      0.59      0.70      1022

    accuracy                           0.75      2044
   macro avg       0.78      0.75      0.74      2044
weighted avg       0.78      0.75      0.74      2044



# **mpress**

In [35]:
mpress = pd.read_pickle('/kaggle/input/packer-balance-pickle/mpress_balance.pickle')

In [36]:
mpress = mpress.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
mpress = mpress.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = mpress.select_dtypes(include=['object']).columns
mpress = mpress.drop(string_columns, axis=1)

In [37]:
df_7000_mpress = mpress.sample(n=7000, random_state=102)
df_3000_mpress = mpress.drop(df_7000_mpress.index)
data_mpress = df_7000_mpress.sample(n=3500, random_state=102)

In [38]:
#Kiểm tra số lượng mẫu
PM = data_mpress[(data_mpress['malicious'] == True) & (data_mpress['packed'] == True)].shape[0]
PB = data_mpress[(data_mpress['malicious'] == False) & (data_mpress['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_mpress = data_mpress[(data_mpress['malicious'] == True) & (data_mpress['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_mpress = data_mpress.drop(data_PM_mpress.index)

#Số lượng tập train
PM = data_mpress[(data_mpress['malicious'] == True) & (data_mpress['packed'] == True)].shape[0]
PB = data_mpress[(data_mpress['malicious'] == False) & (data_mpress['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y_mpress = data_mpress['malicious'].values
data_mpress = data_mpress.drop(['malicious','packed'], axis=1)

PM = df_3000_mpress[(df_3000_mpress['malicious'] == True) & (df_3000_mpress['packed'] == True)].shape[0]
PB = df_3000_mpress[(df_3000_mpress['malicious'] == False) & (df_3000_mpress['packed'] == True)].shape[0]

df_3000_PM_mpress = df_3000_mpress[(df_3000_mpress['malicious'] == True) & (df_3000_mpress['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_mpress = df_3000_mpress.drop(df_3000_PM_mpress.index)

PM = df_3000_mpress[(df_3000_mpress['malicious'] == True) & (df_3000_mpress['packed'] == True)].shape[0]
PB = df_3000_mpress[(df_3000_mpress['malicious'] == False) & (df_3000_mpress['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 1790 samples
Packed Benign: 1710 samples
Total TRAIN set: 3500 samples
Packed Malware: 1710 samples
Packed Benign: 1710 samples
Total TRAIN set: 3420 samples
Packed Malware: 1477 samples
Packed Benign: 1477 samples
Total TEST set: 2954 samples


In [39]:
# model = SelectFromModel(fsel, prefit=True)

X_train_mpress = data_mpress
X_test_mpress = df_3000_mpress.drop(['malicious','packed'], axis=1)

# data_mpress = data_mpress.loc[:, (data_mpress != 0).any(axis=0)]
x_train_mpress = data_mpress
X_test_mpress = df_3000_mpress.drop(['malicious','packed'], axis=1)
y_train_mpress = y_mpress
y_test_mpress = df_3000_mpress['malicious'].values

In [40]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train, y_train)
random_forest_preds = random_forest.predict(X_test_mpress)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_mpress))

The accuracy of the Random Forests model is :	 0.4766418415707515


In [41]:
mt = confusion_matrix(y_test_mpress, random_forest_preds)
report = classification_report(y_test_mpress, random_forest_preds)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 1086 
True negative : 322 
False positive : 1155 
False negative : 391 
Classification Report:
               precision    recall  f1-score   support

       False       0.45      0.22      0.29      1477
        True       0.48      0.74      0.58      1477

    accuracy                           0.48      2954
   macro avg       0.47      0.48      0.44      2954
weighted avg       0.47      0.48      0.44      2954



In [42]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest_new = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest_new.fit(x_train_mpress, y_train_mpress)
random_forest_preds_new = random_forest_new.predict(X_test)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds_new,y_test))

The accuracy of the Random Forests model is :	 0.8228962818003914


In [43]:
mt = confusion_matrix(y_test, random_forest_preds_new)
report = classification_report(y_test, random_forest_preds_new)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 785 
True negative : 897 
False positive : 125 
False negative : 237 
Classification Report:
               precision    recall  f1-score   support

       False       0.79      0.88      0.83      1022
        True       0.86      0.77      0.81      1022

    accuracy                           0.82      2044
   macro avg       0.83      0.82      0.82      2044
weighted avg       0.83      0.82      0.82      2044



# **obsidium**

In [44]:
obsidium = pd.read_pickle('/kaggle/input/packer-balance-pickle/obsidium_balance.pickle')

In [45]:
obsidium = obsidium.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
obsidium = obsidium.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = obsidium.select_dtypes(include=['object']).columns
obsidium = obsidium.drop(string_columns, axis=1)

In [46]:
df_7000_obsidium = obsidium.sample(n=7000, random_state=102)
df_3000_obsidium = obsidium.drop(df_7000_obsidium.index)
data_obsidium = df_7000_obsidium.sample(n=3500, random_state=102)

In [47]:
#Kiểm tra số lượng mẫu
PM = data_obsidium[(data_obsidium['malicious'] == True) & (data_obsidium['packed'] == True)].shape[0]
PB = data_obsidium[(data_obsidium['malicious'] == False) & (data_obsidium['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_obsidium = data_obsidium[(data_obsidium['malicious'] == True) & (data_obsidium['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_obsidium = data_obsidium.drop(data_PM_obsidium.index)

#Số lượng tập train
PM = data_obsidium[(data_obsidium['malicious'] == True) & (data_obsidium['packed'] == True)].shape[0]
PB = data_obsidium[(data_obsidium['malicious'] == False) & (data_obsidium['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y_obsidium = data_obsidium['malicious'].values
data_obsidium = data_obsidium.drop(['malicious','packed'], axis=1)

PM = df_3000_obsidium[(df_3000_obsidium['malicious'] == True) & (df_3000_obsidium['packed'] == True)].shape[0]
PB = df_3000_obsidium[(df_3000_obsidium['malicious'] == False) & (df_3000_obsidium['packed'] == True)].shape[0]

df_3000_PM_obsidium = df_3000_obsidium[(df_3000_obsidium['malicious'] == True) & (df_3000_obsidium['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_obsidium = df_3000_obsidium.drop(df_3000_PM_obsidium.index)

PM = df_3000_obsidium[(df_3000_obsidium['malicious'] == True) & (df_3000_obsidium['packed'] == True)].shape[0]
PB = df_3000_obsidium[(df_3000_obsidium['malicious'] == False) & (df_3000_obsidium['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2306 samples
Packed Benign: 1194 samples
Total TRAIN set: 3500 samples
Packed Malware: 1194 samples
Packed Benign: 1194 samples
Total TRAIN set: 2388 samples
Packed Malware: 1058 samples
Packed Benign: 1058 samples
Total TEST set: 2116 samples


In [48]:
# model = SelectFromModel(fsel, prefit=True)

X_train_obsidium = data_obsidium
X_test_obsidium = df_3000_obsidium.drop(['malicious','packed'], axis=1)

# data_obsidium = data_obsidium.loc[:, (data_obsidium != 0).any(axis=0)]
x_train_obsidium = data_obsidium
X_test_obsidium = df_3000_obsidium.drop(['malicious','packed'], axis=1)
y_train_obsidium = y_obsidium
y_test_obsidium = df_3000_obsidium['malicious'].values

In [49]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train, y_train)
random_forest_preds = random_forest.predict(X_test_obsidium)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_obsidium))

The accuracy of the Random Forests model is :	 0.5600189035916824


In [50]:
mt = confusion_matrix(y_test_obsidium, random_forest_preds)
report = classification_report(y_test_obsidium, random_forest_preds)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 1058 
True negative : 127 
False positive : 931 
False negative : 0 
Classification Report:
               precision    recall  f1-score   support

       False       1.00      0.12      0.21      1058
        True       0.53      1.00      0.69      1058

    accuracy                           0.56      2116
   macro avg       0.77      0.56      0.45      2116
weighted avg       0.77      0.56      0.45      2116



In [51]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest_new = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest_new.fit(x_train_obsidium, y_train_obsidium)
random_forest_preds_new = random_forest_new.predict(X_test)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds_new,y_test))

The accuracy of the Random Forests model is :	 0.8395303326810176


In [52]:
mt = confusion_matrix(y_test, random_forest_preds_new)
report = classification_report(y_test, random_forest_preds_new)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 759 
True negative : 957 
False positive : 65 
False negative : 263 
Classification Report:
               precision    recall  f1-score   support

       False       0.78      0.94      0.85      1022
        True       0.92      0.74      0.82      1022

    accuracy                           0.84      2044
   macro avg       0.85      0.84      0.84      2044
weighted avg       0.85      0.84      0.84      2044



# **pe compact**

In [53]:
pecompact = pd.read_pickle('/kaggle/input/packer-balance-pickle/pecompact_balance.pickle')

In [54]:
pecompact = pecompact.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
pecompact = pecompact.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = pecompact.select_dtypes(include=['object']).columns
pecompact = pecompact.drop(string_columns, axis=1)

In [55]:
df_7000_pecompact = pecompact.sample(n=7000, random_state=102)
df_3000_pecompact = pecompact.drop(df_7000_pecompact.index)
data_pecompact = df_7000_pecompact.sample(n=3500, random_state=102)

In [56]:
#Kiểm tra số lượng mẫu
PM = data_pecompact[(data_pecompact['malicious'] == True) & (data_pecompact['packed'] == True)].shape[0]
PB = data_pecompact[(data_pecompact['malicious'] == False) & (data_pecompact['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_pecompact = data_pecompact[(data_pecompact['malicious'] == True) & (data_pecompact['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_pecompact = data_pecompact.drop(data_PM_pecompact.index)

#Số lượng tập train
PM = data_pecompact[(data_pecompact['malicious'] == True) & (data_pecompact['packed'] == True)].shape[0]
PB = data_pecompact[(data_pecompact['malicious'] == False) & (data_pecompact['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y_pecompact = data_pecompact['malicious'].values
data_pecompact = data_pecompact.drop(['malicious','packed'], axis=1)

PM = df_3000_pecompact[(df_3000_pecompact['malicious'] == True) & (df_3000_pecompact['packed'] == True)].shape[0]
PB = df_3000_pecompact[(df_3000_pecompact['malicious'] == False) & (df_3000_pecompact['packed'] == True)].shape[0]

df_3000_PM_pecompact = df_3000_pecompact[(df_3000_pecompact['malicious'] == True) & (df_3000_pecompact['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_pecompact = df_3000_pecompact.drop(df_3000_PM_pecompact.index)

PM = df_3000_pecompact[(df_3000_pecompact['malicious'] == True) & (df_3000_pecompact['packed'] == True)].shape[0]
PB = df_3000_pecompact[(df_3000_pecompact['malicious'] == False) & (df_3000_pecompact['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2951 samples
Packed Benign: 549 samples
Total TRAIN set: 3500 samples
Packed Malware: 549 samples
Packed Benign: 549 samples
Total TRAIN set: 1098 samples
Packed Malware: 499 samples
Packed Benign: 499 samples
Total TEST set: 998 samples


In [57]:
# model = SelectFromModel(fsel, prefit=True)

X_train_pecompact = data_pecompact
X_test_pecompact = df_3000_pecompact.drop(['malicious','packed'], axis=1)

# data_pecompact = data_pecompact.loc[:, (data_pecompact != 0).any(axis=0)]
x_train_pecompact = data_pecompact
X_test_pecompact = df_3000_pecompact.drop(['malicious','packed'], axis=1)
y_train_pecompact = y_pecompact
y_test_pecompact = df_3000_pecompact['malicious'].values

In [58]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train, y_train)
random_forest_preds = random_forest.predict(X_test_pecompact)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_pecompact))

The accuracy of the Random Forests model is :	 0.7084168336673347


In [59]:
mt = confusion_matrix(y_test_pecompact, random_forest_preds)
report = classification_report(y_test_pecompact, random_forest_preds)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 490 
True negative : 217 
False positive : 282 
False negative : 9 
Classification Report:
               precision    recall  f1-score   support

       False       0.96      0.43      0.60       499
        True       0.63      0.98      0.77       499

    accuracy                           0.71       998
   macro avg       0.80      0.71      0.68       998
weighted avg       0.80      0.71      0.68       998



In [60]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest_new = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest_new.fit(x_train_pecompact, y_train_pecompact)
random_forest_preds_new = random_forest_new.predict(X_test)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds_new,y_test))

The accuracy of the Random Forests model is :	 0.8243639921722113


In [61]:
mt = confusion_matrix(y_test, random_forest_preds_new)
report = classification_report(y_test, random_forest_preds_new)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 875 
True negative : 810 
False positive : 212 
False negative : 147 
Classification Report:
               precision    recall  f1-score   support

       False       0.85      0.79      0.82      1022
        True       0.80      0.86      0.83      1022

    accuracy                           0.82      2044
   macro avg       0.83      0.82      0.82      2044
weighted avg       0.83      0.82      0.82      2044



# **pelock**

In [62]:
pelock = pd.read_pickle('/kaggle/input/packer-balance-pickle/pelock_balance.pickle')

In [63]:
pelock = pelock.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
pelock = pelock.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = pelock.select_dtypes(include=['object']).columns
pelock = pelock.drop(string_columns, axis=1)

In [64]:
df_7000_pelock = pelock.sample(n=7000, random_state=102)
df_3000_pelock = pelock.drop(df_7000_pelock.index)
data_pelock = df_7000_pelock.sample(n=3500, random_state=102)

In [65]:
#Kiểm tra số lượng mẫu
PM = data_pelock[(data_pelock['malicious'] == True) & (data_pelock['packed'] == True)].shape[0]
PB = data_pelock[(data_pelock['malicious'] == False) & (data_pelock['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_pelock = data_pelock[(data_pelock['malicious'] == True) & (data_pelock['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_pelock = data_pelock.drop(data_PM_pelock.index)

#Số lượng tập train
PM = data_pelock[(data_pelock['malicious'] == True) & (data_pelock['packed'] == True)].shape[0]
PB = data_pelock[(data_pelock['malicious'] == False) & (data_pelock['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y_pelock = data_pelock['malicious'].values
data_pelock = data_pelock.drop(['malicious','packed'], axis=1)

PM = df_3000_pelock[(df_3000_pelock['malicious'] == True) & (df_3000_pelock['packed'] == True)].shape[0]
PB = df_3000_pelock[(df_3000_pelock['malicious'] == False) & (df_3000_pelock['packed'] == True)].shape[0]

df_3000_PM_pelock = df_3000_pelock[(df_3000_pelock['malicious'] == True) & (df_3000_pelock['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_pelock = df_3000_pelock.drop(df_3000_PM_pelock.index)

PM = df_3000_pelock[(df_3000_pelock['malicious'] == True) & (df_3000_pelock['packed'] == True)].shape[0]
PB = df_3000_pelock[(df_3000_pelock['malicious'] == False) & (df_3000_pelock['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 1960 samples
Packed Benign: 1540 samples
Total TRAIN set: 3500 samples
Packed Malware: 1540 samples
Packed Benign: 1540 samples
Total TRAIN set: 3080 samples
Packed Malware: 1360 samples
Packed Benign: 1360 samples
Total TEST set: 2720 samples


In [66]:
# model = SelectFromModel(fsel, prefit=True)

X_train_pelock = data_pelock
X_test_pelock = df_3000_pelock.drop(['malicious','packed'], axis=1)

# data_pelock = data_pelock.loc[:, (data_pelock != 0).any(axis=0)]
x_train_pelock = data_pelock
X_test_pelock = df_3000_pelock.drop(['malicious','packed'], axis=1)
y_train_pelock = y_pelock
y_test_pelock = df_3000_pelock['malicious'].values

In [67]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train, y_train)
random_forest_preds = random_forest.predict(X_test_pelock)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_pelock))

The accuracy of the Random Forests model is :	 0.6014705882352941


In [68]:
mt = confusion_matrix(y_test_pelock, random_forest_preds)
report = classification_report(y_test_pelock, random_forest_preds)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 1359 
True negative : 277 
False positive : 1083 
False negative : 1 
Classification Report:
               precision    recall  f1-score   support

       False       1.00      0.20      0.34      1360
        True       0.56      1.00      0.71      1360

    accuracy                           0.60      2720
   macro avg       0.78      0.60      0.53      2720
weighted avg       0.78      0.60      0.53      2720



In [69]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest_new = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest_new.fit(x_train_pelock, y_train_pelock)
random_forest_preds_new = random_forest_new.predict(X_test)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds_new,y_test))

The accuracy of the Random Forests model is :	 0.8571428571428571


In [70]:
mt = confusion_matrix(y_test, random_forest_preds_new)
report = classification_report(y_test, random_forest_preds_new)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 874 
True negative : 878 
False positive : 144 
False negative : 148 
Classification Report:
               precision    recall  f1-score   support

       False       0.86      0.86      0.86      1022
        True       0.86      0.86      0.86      1022

    accuracy                           0.86      2044
   macro avg       0.86      0.86      0.86      2044
weighted avg       0.86      0.86      0.86      2044



# **petite**

In [71]:
petite = pd.read_pickle('/kaggle/input/packer-balance-pickle/petite_balance.pickle')

In [72]:
petite = petite.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
petite = petite.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = petite.select_dtypes(include=['object']).columns
petite = petite.drop(string_columns, axis=1)

In [73]:
df_7000_petite = petite.sample(n=7000, random_state=102)
df_3000_petite = petite.drop(df_7000_petite.index)
data_petite = df_7000_petite.sample(n=3500, random_state=102)

In [74]:
#Kiểm tra số lượng mẫu
PM = data_petite[(data_petite['malicious'] == True) & (data_petite['packed'] == True)].shape[0]
PB = data_petite[(data_petite['malicious'] == False) & (data_petite['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_petite = data_petite[(data_petite['malicious'] == True) & (data_petite['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_petite = data_petite.drop(data_PM_petite.index)

#Số lượng tập train
PM = data_petite[(data_petite['malicious'] == True) & (data_petite['packed'] == True)].shape[0]
PB = data_petite[(data_petite['malicious'] == False) & (data_petite['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y_petite = data_petite['malicious'].values
data_petite = data_petite.drop(['malicious','packed'], axis=1)

PM = df_3000_petite[(df_3000_petite['malicious'] == True) & (df_3000_petite['packed'] == True)].shape[0]
PB = df_3000_petite[(df_3000_petite['malicious'] == False) & (df_3000_petite['packed'] == True)].shape[0]

df_3000_PM_petite = df_3000_petite[(df_3000_petite['malicious'] == True) & (df_3000_petite['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_petite = df_3000_petite.drop(df_3000_PM_petite.index)

PM = df_3000_petite[(df_3000_petite['malicious'] == True) & (df_3000_petite['packed'] == True)].shape[0]
PB = df_3000_petite[(df_3000_petite['malicious'] == False) & (df_3000_petite['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2302 samples
Packed Benign: 1198 samples
Total TRAIN set: 3500 samples
Packed Malware: 1198 samples
Packed Benign: 1198 samples
Total TRAIN set: 2396 samples
Packed Malware: 1056 samples
Packed Benign: 1056 samples
Total TEST set: 2112 samples


In [75]:
# model = SelectFromModel(fsel, prefit=True)

X_train_petite = data_petite
X_test_petite = df_3000_petite.drop(['malicious','packed'], axis=1)

# data_petite = data_petite.loc[:, (data_petite != 0).any(axis=0)]
x_train_petite = data_petite
X_test_petite = df_3000_petite.drop(['malicious','packed'], axis=1)
y_train_petite = y_petite
y_test_petite = df_3000_petite['malicious'].values

In [76]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train, y_train)
random_forest_preds = random_forest.predict(X_test_petite)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_petite))

The accuracy of the Random Forests model is :	 0.5861742424242424


In [77]:
mt = confusion_matrix(y_test_petite, random_forest_preds)
report = classification_report(y_test_petite, random_forest_preds)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 1052 
True negative : 186 
False positive : 870 
False negative : 4 
Classification Report:
               precision    recall  f1-score   support

       False       0.98      0.18      0.30      1056
        True       0.55      1.00      0.71      1056

    accuracy                           0.59      2112
   macro avg       0.76      0.59      0.50      2112
weighted avg       0.76      0.59      0.50      2112



In [78]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest_new = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest_new.fit(x_train_petite, y_train_petite)
random_forest_preds_new = random_forest_new.predict(X_test)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds_new,y_test))

The accuracy of the Random Forests model is :	 0.8532289628180039


In [79]:
mt = confusion_matrix(y_test, random_forest_preds_new)
report = classification_report(y_test, random_forest_preds_new)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 882 
True negative : 862 
False positive : 160 
False negative : 140 
Classification Report:
               precision    recall  f1-score   support

       False       0.86      0.84      0.85      1022
        True       0.85      0.86      0.85      1022

    accuracy                           0.85      2044
   macro avg       0.85      0.85      0.85      2044
weighted avg       0.85      0.85      0.85      2044



# **telock**

In [80]:
telock = pd.read_pickle('/kaggle/input/packer-balance-pickle/telock_balance.pickle')

In [81]:
telock = telock.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
telock = telock.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = telock.select_dtypes(include=['object']).columns
telock = telock.drop(string_columns, axis=1)

In [82]:
df_7000_telock = telock.sample(n=7000, random_state=102)
df_3000_telock = telock.drop(df_7000_telock.index)
data_telock = df_7000_telock.sample(n=3500, random_state=102)

In [83]:
#Kiểm tra số lượng mẫu
PM = data_telock[(data_telock['malicious'] == True) & (data_telock['packed'] == True)].shape[0]
PB = data_telock[(data_telock['malicious'] == False) & (data_telock['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_telock = data_telock[(data_telock['malicious'] == True) & (data_telock['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_telock = data_telock.drop(data_PM_telock.index)

#Số lượng tập train
PM = data_telock[(data_telock['malicious'] == True) & (data_telock['packed'] == True)].shape[0]
PB = data_telock[(data_telock['malicious'] == False) & (data_telock['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y_telock = data_telock['malicious'].values
data_telock = data_telock.drop(['malicious','packed'], axis=1)

PM = df_3000_telock[(df_3000_telock['malicious'] == True) & (df_3000_telock['packed'] == True)].shape[0]
PB = df_3000_telock[(df_3000_telock['malicious'] == False) & (df_3000_telock['packed'] == True)].shape[0]

df_3000_PM_telock = df_3000_telock[(df_3000_telock['malicious'] == True) & (df_3000_telock['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_telock = df_3000_telock.drop(df_3000_PM_telock.index)

PM = df_3000_telock[(df_3000_telock['malicious'] == True) & (df_3000_telock['packed'] == True)].shape[0]
PB = df_3000_telock[(df_3000_telock['malicious'] == False) & (df_3000_telock['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2995 samples
Packed Benign: 505 samples
Total TRAIN set: 3500 samples
Packed Malware: 505 samples
Packed Benign: 505 samples
Total TRAIN set: 1010 samples
Packed Malware: 432 samples
Packed Benign: 432 samples
Total TEST set: 864 samples


In [84]:
# model = SelectFromModel(fsel, prefit=True)

X_train_telock = data_telock
X_test_telock = df_3000_telock.drop(['malicious','packed'], axis=1)

# data_telock = data_telock.loc[:, (data_telock != 0).any(axis=0)]
x_train_telock = data_telock
X_test_telock = df_3000_telock.drop(['malicious','packed'], axis=1)
y_train_telock = y_telock
y_test_telock = df_3000_telock['malicious'].values

In [85]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train, y_train)
random_forest_preds = random_forest.predict(X_test_telock)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_telock))

The accuracy of the Random Forests model is :	 0.5231481481481481


In [86]:
mt = confusion_matrix(y_test_telock, random_forest_preds)
report = classification_report(y_test_telock, random_forest_preds)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 430 
True negative : 22 
False positive : 410 
False negative : 2 
Classification Report:
               precision    recall  f1-score   support

       False       0.92      0.05      0.10       432
        True       0.51      1.00      0.68       432

    accuracy                           0.52       864
   macro avg       0.71      0.52      0.39       864
weighted avg       0.71      0.52      0.39       864



In [87]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest_new = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest_new.fit(x_train_telock, y_train_telock)
random_forest_preds_new = random_forest_new.predict(X_test)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds_new,y_test))

The accuracy of the Random Forests model is :	 0.7191780821917808


In [88]:
mt = confusion_matrix(y_test, random_forest_preds_new)
report = classification_report(y_test, random_forest_preds_new)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 780 
True negative : 690 
False positive : 332 
False negative : 242 
Classification Report:
               precision    recall  f1-score   support

       False       0.74      0.68      0.71      1022
        True       0.70      0.76      0.73      1022

    accuracy                           0.72      2044
   macro avg       0.72      0.72      0.72      2044
weighted avg       0.72      0.72      0.72      2044



# **themida**

In [89]:
themida = pd.read_pickle('/kaggle/input/packer-balance-pickle/themida_balance.pickle')

In [90]:
themida = themida.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
themida = themida.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = themida.select_dtypes(include=['object']).columns
themida = themida.drop(string_columns, axis=1)

In [91]:
df_7000_themida = themida.sample(n=7000, random_state=102)
df_3000_themida = themida.drop(df_7000_themida.index)
data_themida = df_7000_themida.sample(n=3500, random_state=102)

In [92]:
#Kiểm tra số lượng mẫu
PM = data_themida[(data_themida['malicious'] == True) & (data_themida['packed'] == True)].shape[0]
PB = data_themida[(data_themida['malicious'] == False) & (data_themida['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_themida = data_themida[(data_themida['malicious'] == True) & (data_themida['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_themida = data_themida.drop(data_PM_themida.index)

#Số lượng tập train
PM = data_themida[(data_themida['malicious'] == True) & (data_themida['packed'] == True)].shape[0]
PB = data_themida[(data_themida['malicious'] == False) & (data_themida['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y_themida = data_themida['malicious'].values
data_themida = data_themida.drop(['malicious','packed'], axis=1)

PM = df_3000_themida[(df_3000_themida['malicious'] == True) & (df_3000_themida['packed'] == True)].shape[0]
PB = df_3000_themida[(df_3000_themida['malicious'] == False) & (df_3000_themida['packed'] == True)].shape[0]

df_3000_PM_themida = df_3000_themida[(df_3000_themida['malicious'] == True) & (df_3000_themida['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_themida = df_3000_themida.drop(df_3000_PM_themida.index)

PM = df_3000_themida[(df_3000_themida['malicious'] == True) & (df_3000_themida['packed'] == True)].shape[0]
PB = df_3000_themida[(df_3000_themida['malicious'] == False) & (df_3000_themida['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2241 samples
Packed Benign: 1259 samples
Total TRAIN set: 3500 samples
Packed Malware: 1259 samples
Packed Benign: 1259 samples
Total TRAIN set: 2518 samples
Packed Malware: 1089 samples
Packed Benign: 1089 samples
Total TEST set: 2178 samples


In [93]:
# model = SelectFromModel(fsel, prefit=True)

X_train_themida = data_themida
X_test_themida = df_3000_themida.drop(['malicious','packed'], axis=1)

# data_themida = data_themida.loc[:, (data_themida != 0).any(axis=0)]
x_train_themida = data_themida
X_test_themida = df_3000_themida.drop(['malicious','packed'], axis=1)
y_train_themida = y_themida
y_test_themida = df_3000_themida['malicious'].values

In [94]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train, y_train)
random_forest_preds = random_forest.predict(X_test_themida)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_themida))

The accuracy of the Random Forests model is :	 0.8044077134986226


In [95]:
mt = confusion_matrix(y_test_themida, random_forest_preds)
report = classification_report(y_test_themida, random_forest_preds)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 1080 
True negative : 672 
False positive : 417 
False negative : 9 
Classification Report:
               precision    recall  f1-score   support

       False       0.99      0.62      0.76      1089
        True       0.72      0.99      0.84      1089

    accuracy                           0.80      2178
   macro avg       0.85      0.80      0.80      2178
weighted avg       0.85      0.80      0.80      2178



In [96]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest_new = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest_new.fit(x_train_themida, y_train_themida)
random_forest_preds_new = random_forest_new.predict(X_test)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds_new,y_test))

The accuracy of the Random Forests model is :	 0.8370841487279843


In [97]:
mt = confusion_matrix(y_test, random_forest_preds_new)
report = classification_report(y_test, random_forest_preds_new)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 918 
True negative : 793 
False positive : 229 
False negative : 104 
Classification Report:
               precision    recall  f1-score   support

       False       0.88      0.78      0.83      1022
        True       0.80      0.90      0.85      1022

    accuracy                           0.84      2044
   macro avg       0.84      0.84      0.84      2044
weighted avg       0.84      0.84      0.84      2044



# **upx**

In [98]:
upx = pd.read_pickle('/kaggle/input/packer-balance-pickle/upx_balance.pickle')

In [99]:
upx = upx.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
upx = upx.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = upx.select_dtypes(include=['object']).columns
upx = upx.drop(string_columns, axis=1)

In [100]:
df_7000_upx = upx.sample(n=7000, random_state=102)
df_3000_upx = upx.drop(df_7000_upx.index)
data_upx = df_7000_upx.sample(n=3500, random_state=102)

In [101]:
#Kiểm tra số lượng mẫu
PM = data_upx[(data_upx['malicious'] == True) & (data_upx['packed'] == True)].shape[0]
PB = data_upx[(data_upx['malicious'] == False) & (data_upx['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_upx = data_upx[(data_upx['malicious'] == True) & (data_upx['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_upx = data_upx.drop(data_PM_upx.index)

#Số lượng tập train
PM = data_upx[(data_upx['malicious'] == True) & (data_upx['packed'] == True)].shape[0]
PB = data_upx[(data_upx['malicious'] == False) & (data_upx['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y_upx = data_upx['malicious'].values
data_upx = data_upx.drop(['malicious','packed'], axis=1)

PM = df_3000_upx[(df_3000_upx['malicious'] == True) & (df_3000_upx['packed'] == True)].shape[0]
PB = df_3000_upx[(df_3000_upx['malicious'] == False) & (df_3000_upx['packed'] == True)].shape[0]

df_3000_PM_upx = df_3000_upx[(df_3000_upx['malicious'] == True) & (df_3000_upx['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_upx = df_3000_upx.drop(df_3000_PM_upx.index)

PM = df_3000_upx[(df_3000_upx['malicious'] == True) & (df_3000_upx['packed'] == True)].shape[0]
PB = df_3000_upx[(df_3000_upx['malicious'] == False) & (df_3000_upx['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2391 samples
Packed Benign: 1109 samples
Total TRAIN set: 3500 samples
Packed Malware: 1109 samples
Packed Benign: 1109 samples
Total TRAIN set: 2218 samples
Packed Malware: 974 samples
Packed Benign: 974 samples
Total TEST set: 1948 samples


In [102]:
# model = SelectFromModel(fsel, prefit=True)

X_train_upx = data_upx
X_test_upx = df_3000_upx.drop(['malicious','packed'], axis=1)

# data_upx = data_upx.loc[:, (data_upx != 0).any(axis=0)]
x_train_upx = data_upx
X_test_upx = df_3000_upx.drop(['malicious','packed'], axis=1)
y_train_upx = y_upx
y_test_upx = df_3000_upx['malicious'].values

In [103]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train, y_train)
random_forest_preds = random_forest.predict(X_test_upx)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_upx))

The accuracy of the Random Forests model is :	 0.7659137577002053


In [104]:
mt = confusion_matrix(y_test_upx, random_forest_preds)
report = classification_report(y_test_upx, random_forest_preds)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 919 
True negative : 573 
False positive : 401 
False negative : 55 
Classification Report:
               precision    recall  f1-score   support

       False       0.91      0.59      0.72       974
        True       0.70      0.94      0.80       974

    accuracy                           0.77      1948
   macro avg       0.80      0.77      0.76      1948
weighted avg       0.80      0.77      0.76      1948



In [105]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest_new = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest_new.fit(x_train_upx, y_train_upx)
random_forest_preds_new = random_forest_new.predict(X_test)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds_new,y_test))

The accuracy of the Random Forests model is :	 0.8850293542074364


In [106]:
mt = confusion_matrix(y_test, random_forest_preds_new)
report = classification_report(y_test, random_forest_preds_new)
print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))
print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))
print("Classification Report:\n", report)

True positive : 927 
True negative : 882 
False positive : 140 
False negative : 95 
Classification Report:
               precision    recall  f1-score   support

       False       0.90      0.86      0.88      1022
        True       0.87      0.91      0.89      1022

    accuracy                           0.89      2044
   macro avg       0.89      0.89      0.88      2044
weighted avg       0.89      0.89      0.88      2044

